In [1]:
# fmt: off
import logging
from pathlib import Path

from farm.data_handler.data_silo import DataSilo
from farm.data_handler.processor import TextClassificationProcessor
from farm.infer import Inferencer
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.language_model import LanguageModel
from farm.modeling.optimization import initialize_optimizer
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.tokenization import Tokenizer
from farm.train import Trainer
from farm.data_handler.utils import grouper
import functools

from farm.utils import initialize_device_settings, set_all_seeds, calc_chunksize
from jack.logging.module import logger as ai_logger

import warnings
warnings.filterwarnings('ignore')

08/30/2022 16:19:11 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
##########################
########## Settings
##########################
set_all_seeds(seed=2077)
n_epochs = 10
batch_size = 4
evaluate_every = 150
lang_model = "cointegrated/rubert-tiny"
do_lower_case = False
dev_split = 0.0
dev_stratification = True
max_processes = 1    # 128 is default
max_chunksize = 512
# or a local path:
# lang_model = Path("../saved_models/farm-bert-base-cased")
use_amp = None

device, n_gpu = initialize_device_settings(use_cuda=True, use_amp=use_amp)

08/30/2022 16:19:14 - INFO - farm.utils -   Using device: CUDA 
08/30/2022 16:19:14 - INFO - farm.utils -   Number of GPUs: 1
08/30/2022 16:19:14 - INFO - farm.utils -   Distributed Training: False
08/30/2022 16:19:14 - INFO - farm.utils -   Automatic Mixed Precision: None


In [3]:
# 1.Create a tokenizer
tokenizer = Tokenizer.load(pretrained_model_name_or_path=lang_model, do_lower_case=do_lower_case, use_fast=True)

08/30/2022 16:19:16 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'BertTokenizer'


In [4]:
labels = [
    "Вопросы по порядку начисления процентов/размере процентной ставки по накопительному счету или счету \"Активный возраст\" (почему такая ставка/почему уменьшается ставка/как начисляются проценты/как начисляются проценты при досрочном закрытии вклада/не согласен с начисленными процентами, но проценты начислены корректно и пояснен порядок начисления %)",
    "Подбор вклада/условия вклада, который планирует открыть (какие вклады есть в СБ/какой вклад выгодней/размер ставки/какой вклад подойдет для совершения определенной операции/условия вклада по поступившему предложению от банка/какие приходные или расходные операции доступны/ограничения по сумме открытия или пополнения)",
    "Способы или сроки закрытия вклада/можно ли закрыть вклад досрочно (в том числе, как закрыть в СБОЛ/МП/как закрыть валютный вклад без конвертации)",
    "Условия/порядок проведений операций по действующему вкладу (срок вклада/сколько средств должно храниться на вкладе, чтобы он не был закрыт/какие доступны операции/где можно снять средства со вклада/какие документы нужны для проведения операций по вкладу)",
    "Порядок начисления процентов по вкладам, кроме Накопительного счета и Активный возраст (когда или как начисляются проценты/периодичность капитализации/как начисляются проценты при досрочном закрытии вклада/какой процент будет при дополнительном взносе/не согласен с начисленными процентами, но проценты начислены корректно и пояснен порядок начисления)",
    "Порядок пролонгации вклада (нужно ли обращаться в ВСП для продления вклада/продлевается ли вклад автоматически/можно ли досрочно продлить вклад/можно ли не продлевать вклад/по какой ставке произойдет пролонгация вклада/меняются ли реквизиты при пролонгации/на какую сумму пролонгируется вклад, с учетом или без учета уже начисленных процентов/в какое время произойдет пролонгация)",
    "Порядок или способы открытия вклада/счета (как или где открыть/кто может открыть/какие документы нужны для открытия/пояснения по интерфейсу СБОЛ или МП при открытии)",
    "Как/когда возможно получить проценты или закрыть вклад, чтобы не потерять начисленные проценты (в т.ч. когда дата приходится на выходной/праздничный день)",
    "UNKNOWN",
]

In [5]:
label_list = labels
metric = "f1_micro"

In [23]:
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=192,
                                        data_dir=Path.home() / "Dataset" / "rc",
                                        train_filename="x_july.csv",
                                        dev_filename="y_july.csv",
                                        test_filename="y_july.csv",
                                        label_list=label_list,
                                        metric=metric,
                                        dev_split=dev_split,
                                        delimiter=",",
                                        dev_stratification=dev_stratification,
                                        text_column_name="text",
                                        label_column_name="label"
                                        )

In [24]:
# 3. Create a DataSilo that loads several datasets (train/dev/test), provides DataLoaders for them and calculates a
#    few descriptive statistics of our datasets
data_silo = DataSilo(
        processor=processor,
        max_processes=max_processes,
        batch_size=batch_size
)

08/30/2022 16:29:30 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
08/30/2022 16:29:30 - INFO - farm.data_handler.data_silo -   LOADING TRAIN DATA
08/30/2022 16:29:30 - INFO - farm.data_handler.data_silo -   ==================
08/30/2022 16:29:30 - INFO - farm.data_handler.data_silo -   Loading train set from: /home/justatom/Dataset/rc/x_july.csv 
08/30/2022 16:29:30 - INFO - farm.data_handler.data_silo -   Multiprocessing disabled, using a single worker to convert 3737 dictionaries to pytorch datasets.
Preprocessing Dataset /home/justatom/Dataset/rc/x_july.csv:   0%|          | 0/3737 [00:00<?, ? Dicts/s]08/30/2022 16:29:31 - INFO - farm.data_handler.processor -   *** Show 1 random examples ***
08/30/2022 16:29:31 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
   

In [18]:
# processor.file_to_dicts(Path.home() / "Dataset" / "rc" / "x_july.csv")

In [8]:
dicts = processor.file_to_dicts(Path.home() / "Dataset" / "rc" / "x_july.csv")

In [9]:
num_dicts = len(dicts)

In [10]:
multiprocessing_chunk_size, num_cpus_used = calc_chunksize(
            num_dicts=num_dicts,
            max_processes=max_processes,
            max_chunksize=max_chunksize,
        )

In [11]:
num_cpus_used

1

In [12]:
def _dataset_from_chunk(chunk, processor):
        """
        Creating a dataset for a chunk (= subset) of dicts. In multiprocessing:
          * we read in all dicts from a file
          * split all dicts into chunks
          * feed *one chunk* to *one process*
          => the *one chunk*  gets converted to *one dataset* (that's what we do here)
          * all datasets get collected and concatenated
        :param chunk: Instead of only having a list of dicts here we also supply an index (ascending int) for each.
            => [(0, dict), (1, dict) ...]
        :type chunk: list of tuples
        :param processor: FARM Processor (e.g. TextClassificationProcessor)
        :return: PyTorch Dataset
        """
        dicts = [d[1] for d in chunk]
        indices = [x[0] for x in chunk]
        dataset, tensor_names, problematic_sample_ids = processor.dataset_from_dicts(dicts=dicts, indices=indices)
        return dataset, tensor_names, problematic_sample_ids

In [32]:
from contextlib import ExitStack

In [33]:
with ExitStack() as stack:
    results = map(functools.partial(_dataset_from_chunk, processor=processor), grouper(dicts, num_dicts), chunksize=1)

In [35]:
datasets = []

In [36]:
for dataset, tensor_names, problematic_samples in results:
    datasets.append(dataset)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [25]:
loss_fn = "crossentropy"

In [26]:
# 4. Create an AdaptiveModel
# a) which consists of a pretrained language model as a basis
language_model = LanguageModel.load(lang_model)
# b) and a prediction head on top that is suited for our task => Text classification
prediction_head = TextClassificationHead(
    class_weights=data_silo.calculate_class_weights(task_name="text_classification"),
    num_labels=len(label_list),
    loss_fn=loss_fn
)

08/30/2022 16:30:23 - INFO - farm.modeling.language_model -   
08/30/2022 16:30:23 - INFO - farm.modeling.language_model -   LOADING MODEL
08/30/2022 16:30:23 - INFO - farm.modeling.language_model -   =============
08/30/2022 16:30:23 - INFO - farm.modeling.language_model -   Could not find cointegrated/rubert-tiny locally.
08/30/2022 16:30:23 - INFO - farm.modeling.language_model -   Looking on Transformers Model Hub (in local cache and online)...
Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with 

In [27]:
model = AdaptiveModel(
        language_model=language_model,
        prediction_heads=[prediction_head],
        embeds_dropout_prob=0.1,
        lm_output_types=["per_sequence"],
        device=device)

08/30/2022 16:30:26 - INFO - farm.modeling.prediction_head -   Resizing input dimensions of TextClassificationHead (text_classification) from [768, 9] to [312, 9] to match language model


In [28]:
# 5. Create an optimizer
model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    learning_rate=3e-5,
    device=device,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=n_epochs,
    use_amp=use_amp)

08/30/2022 16:30:28 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 3e-05}'
08/30/2022 16:30:28 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_with_warmup'
08/30/2022 16:30:28 - INFO - farm.modeling.optimization -   Loading schedule `get_linear_schedule_with_warmup`: '{'num_warmup_steps': 935.0, 'num_training_steps': 9350}'


In [29]:
API_KEY = "9b7524ccc0cc7f67444fa6d0662c993fba1dde33"
project_name = "RC"
experiment_name = "rc_july"

In [30]:
# 6a. Create logger to enable finetuning. Uncomment lines below and initialize a logger
ml_logger = ai_logger.WANDBLogger.init_experiment(
    project_name=project_name,
    experiment_name=experiment_name,
    prefix=f"{loss_fn} # ",
    api=API_KEY,
    sync_step=False,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
# 6. Feed everything to the Trainer, which keeps care of growing our model into powerful plant and evaluates it from time to time
trainer = Trainer(
    prefix="",
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=n_epochs,
    n_gpu=n_gpu,
    lr_schedule=lr_schedule,
    log_loss_every=1,
    evaluate_every=evaluate_every,
    tracker=ml_logger,
    device=device)

In [32]:
# 7. Let it grow
trainer.train()

Evaluating: 100%|██████████| 245/245 [00:00<00:00, 296.73it/s]5/935 [00:03<00:15, 49.71it/s]
08/30/2022 16:30:52 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | D.E.V. SET | AFTER 150 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

08/30/2022 16:30:52 - INFO - farm.eval -   
 _________ text_classification _________
08/30/2022 16:30:52 - INFO - farm.eval -   -loss: 2.1637647203193593
08/30/2022 16:30:52 - INFO - farm.eval -   task_name: text_classification
08/30/2022 16:30:52 - INFO - farm.eval -   f1_micro: 0.17604912998976457
08/30/2022 16:30:52 - INFO - farm.eval -   report: 
 {'Вопросы по порядку начисления процентов/размере процентной ставки по накопительному счету или счету "Активный возраст" (почему такая 

AdaptiveModel(
  (language_model): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(29564, 312, padding_idx=0)
        (position_embeddings): Embedding(512, 312)
        (token_type_embeddings): Embedding(2, 312)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=312, out_features=312, bias=True)
                (key): Linear(in_features=312, out_features=312, bias=True)
                (value): Linear(in_features=312, out_features=312, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=312, out_features=312, bias=True)
            

In [33]:
save_dir = Path.home() / "Weights" / "coi" / "crossentropy"

In [34]:
model.save(save_dir)

In [35]:
processor.save(save_dir)

In [36]:
ml_logger.end_run()

crossentropy # D.E.V._-loss_text_classification,▇▇▆▄▂▂▂▁▂▁▁▁▁▂▁▁▂▃▂▂▂▃▄▃▃▄▅▅▅▆▆▆▆▇▇▇▇███
crossentropy # D.E.V._f1_micro_text_classification,▁▅▅▆▇▆▆▇▇▇▇█▇████████▇████▇███▇▇▇█▇▇██▇▇
crossentropy # T.E.S.T._-loss_text_classification,▁
crossentropy # T.E.S.T._f1_micro_text_classification,▁
crossentropy # TrainLoss,▃▄▅▃▃▆▆▅▃▃▂▂▅▅▂▂▄▅█▅▃▁▂▄▂▁▁▂▁▃▁▂▁▁▂▁▁▁▁▁
x,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
crossentropy # D.E.V._-loss_text_classification,2.25921
crossentropy # D.E.V._f1_micro_text_classification,0.50051
crossentropy # T.E.S.T._-loss_text_classification,2.25824
crossentropy # T.E.S.T._f1_micro_text_classification,0.50051
crossentropy # TrainLoss,0.01197
